In [11]:
#!pip install sentence_transformers
import os
import openai

#openai.api_key  = os.environ['OPENAI_API_KEY']

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key  = os.environ['OPENAI_API_KEY']

In [1]:
from gradio_client import Client

client = Client("https://ysharma-explore-llamav2-with-tgi.hf.space/")
result = client.predict(
				"Howdy!",	# str in 'Message' Textbox component
				api_name="/chat"
)
print(result)

c:\Users\anton_glfk00f\miniconda3\envs\llm_app_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded as API: https://ysharma-explore-llamav2-with-tgi.hf.space/ ✔
 Hello


In [8]:
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

reader = PdfReader("PROGRAMA ANALITICO MÓDULO.pdf")
#reader = PdfReader("PatekPhilippe_P2400_Manual_Winding.pdf")
#reader = PdfReader("Philip Kotler, Gary Armstrong - Marketing_ Versión latinoamerica (2007).pdf")
#reader = PdfReader("Philip Kotler, Gary Armstrong, Sridhar Balasubramanian - Principles of Marketing-Pearson (2023).pdf")

text = ""
i = 0

for page in reader.pages:
    i += 1
    text += f"Página no. {i}: " + page.extract_text()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    add_start_index = True,
    )
chunks = text_splitter.split_text(text=text)

In [1]:
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

reader = PdfReader("PROGRAMA ANALITICO MÓDULO.pdf")
#reader = PdfReader("PatekPhilippe_P2400_Manual_Winding.pdf")
#reader = PdfReader("Philip Kotler, Gary Armstrong - Marketing_ Versión latinoamerica (2007).pdf")
#reader = PdfReader("Philip Kotler, Gary Armstrong, Sridhar Balasubramanian - Principles of Marketing-Pearson (2023).pdf")


embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v1")
VectorStore = FAISS.from_texts([""], embedding=embeddings)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    add_start_index = False,
    )
i = 0
for page in reader.pages:
    i += 1
    text = page.extract_text()
    #print(text)
    chunks = text_splitter.split_text(text=text)
    chunks_pages = []
    for chunk in chunks:
        chunk_wpage = f"Página no. {i} " + chunk
        chunks_pages.append(chunk_wpage)
    print(chunks_pages)
    VectorStore.add_texts(chunks_pages)

c:\Users\anton_glfk00f\miniconda3\envs\llm_app_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['Página no. 1 UAGRM  Business School    Calle Gral. Saavedra Nº 137     Teléfono  3 -366814      www.uagrmbs.edu.bo   \n  E-mail: solicitudes@uagrmbs.edu.bo   \n  \n   ESPECIFICACION  \n \nREQUISITOS ACADEMICOS PARA LA REALIZACION DE  \nUN  MODULO  \nCódigo: ES -ENA -551  Rev.   6, Fecha:  23-03-12  \nPág.  1 de 4  \n  \n   \n \n  \nUAGRM BUSINESS SCHOOL      \n  \n  \n \n  \n   \n  \n  \nMAESTRÍA:  \nMAESTRÍA EN ADMINISTRACIÓN DE EMPRESAS  \n(Versión 9º – Edición 10º)  \nGESTIÓN DE MARKETING   \n(ESTRATÉGICO Y OPERATIVO)  \n  \n  \n \n  \nEnero de 2022  \nSanta Cruz – Bolivia  \n \n \nANEXO  \n  \n1 \n: \n  \nFORMATO DE  \nPRESENTACION DEL PROGRAMA ANAL  \nI \nTICO  \n:']
['Página no. 2 UAGRM  Business School    Calle Gral. Saavedra Nº 137     Teléfono  3 -366814      www.uagrmbs.edu.bo   \n  E-mail: solicitudes@uagrmbs.edu.bo   \n  \n   ESPECIFICACION  \n \nREQUISITOS ACADEMICOS PARA LA REALIZACION DE  \nUN  MODULO  \nCódigo: ES -ENA -551  Rev.   6, Fecha:  23-03-12  \nPág.  2 de 4 

In [88]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v1")
VectorStore = FAISS.from_texts(chunks, embedding=embeddings)

In [24]:
import pickle

# Save the object to a file using pickle
file_name = 'VectorStore_Marketing_en.pickle'
with open(file_name, 'wb') as file:
    pickle.dump(VectorStore, file)


In [25]:
import pickle

# Load the object from the file using pickle
with open('VectorStore_Marketing.pickle', 'rb') as file:
    loaded_VectorStore = pickle.load(file)

In [3]:
from gpt4all import GPT4All
model = GPT4All("llama-2-7b-chat.ggmlv3.q4_0.bin")
output = model.generate("The capital of France is ", max_tokens=30)
print(output)

Found model file at  C:\\\\Users\\\\anton_glfk00f\\\\.cache\\\\gpt4all\\llama-2-7b-chat.ggmlv3.q4_0.bin
 Paris, which is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre


In [40]:
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from gradio_client import Client

class LLM_gradioAPI(LLM):
    n: int

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        # Remove the restriction on the stop parameter
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")


        # Make the API call using the Gradio Client
        client = Client("https://ysharma-explore-llamav2-with-tgi.hf.space/")
        result = client.predict(prompt, api_name="/chat_1")

        # Return the response from the API
        return result

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"n": self.n}

In [18]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback
from langchain.llms import GPT4All
from src import LLM_gradioAPI

query = "Responde en español: ¿Cuál es la experiencia  de la docente?"
docs = VectorStore.similarity_search(query=query, k=3)
#docs = VectorStore.max_marginal_relevance_search(query=query, k=5)

#llm = OpenAI()
#llm = GPT4All(model="./llama-2-7b-chat.ggmlv3.q4_0.bin",max_tokens=2048)
llm = LLM_gradioAPI(n=100) # hugging face repo
chain = load_qa_chain(llm=llm, chain_type="stuff", verbose=True)

reference = "------Also, Write in new lines the initial 10 words of each context for reference."
with get_openai_callback() as cb:
    response = chain.run(input_documents=docs, question=query)
    print(cb)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Página no. 2 Experiencia en el área comercial y ventas de empresas de servicio como Principal Créditos Hipotecarios, 
Andueza Corredores de Seguros, Santiago -Chile. Gerencia de Ventas  de Joyerías Anahí, Santa 
CruzBolivia. Consultoría en temas empresariales y de marketing. Docencia en pre y postgrado  en UTEPSA, 
UAGRM Business School y UCB.  
  
II. PRESENTACION DEL MODULO  
El módulo Gestión de Marketing entrega la metodología y las herramientas para entender el mercado en el 
cual está compiten las  empresas, comprendiendo la importancia del entorno y de las habilidades de la gestión 
de marketing para aprovechas las oportunid ades y defenderse de las amenazas.   
El postgraduante podrá incrementar su habilidad de a

In [19]:
response

'La docente tiene experiencia en el área comercial y ventas de empresas de servicio, como Principal Créditos Hipotecarios y Andueza Corredores de Seguros, en Santiago, Chile. También ha trabajado en la gerencia de ventas de Joyerías Anahí en Santa Cruz, Bolivia. Además, ha sido consultora en temas empresariales y de marketing, y ha impartido clases en pre y postgrados en UTEPSA, UAGRM Business School y UCB.'

In [106]:
response

' The teacher has experience in marketing management and sales of financial institutions, credit agencies, Santiago - Chile. Marketing consulting services, business school and UCB.'

In [33]:
response

' Las estrategias del marketing incluyen estrategias específicas para mercados meta, posicionamiento, la mezcla de marketing y los niveles de los gastos de marketing.'

In [26]:
response

' El marketing es el proceso de satisfacer las necesidades de los consumidores mediante la identificación, anticipación y satisfacción de sus deseos. Esto implica determinar qué productos y servicios los consumidores desean, establecer un precio adecuado para ellos, promocionarlos adecuadamente y distribuirlos en el lugar correcto. El marketing involucra también el estudio de la competencia y el análisis de la industria.'

In [21]:
response

' El marketing es el proceso de diseñar relaciones redituables con el cliente mediante la creación de valor para los clientes y la captación de valor de éstos a cambio. Se enfoca en la creación de valor para los clientes a través de la identificación y satisfacción de sus necesidades. Esto se lleva a cabo mediante la promoción, ventas y distribución de productos y servicios.'

### Chain

In [47]:
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings # see also sentence-transformers
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback
from src import LLM_gradioAPI

reader = PdfReader("PROGRAMA ANALITICO MÓDULO.pdf")
#reader = PdfReader("PatekPhilippe_P2400_Manual_Winding.pdf")
#reader = PdfReader("Philip Kotler, Gary Armstrong - Marketing_ Versión latinoamerica (2007).pdf")
#reader = PdfReader("Philip Kotler, Gary Armstrong, Sridhar Balasubramanian - Principles of Marketing-Pearson (2023).pdf")


embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v1")
VectorStore = FAISS.from_texts([""], embedding=embeddings)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    add_start_index = False,
    )
i = 0
for page in reader.pages:
    i += 1
    text = page.extract_text()
    #print(page)
    chunks = text_splitter.split_text(text=text)
    chunks_pages = []
    for chunk in chunks:
        chunk_wpage = f"Página no. {i} " + chunk
        chunks_pages.append(chunk_wpage)
    print(chunks_pages)
    VectorStore.add_texts(chunks_pages)

['Página no. 1 UAGRM  Business School    Calle Gral. Saavedra Nº 137     Teléfono  3 -366814      www.uagrmbs.edu.bo   \n  E-mail: solicitudes@uagrmbs.edu.bo   \n  \n   ESPECIFICACION  \n \nREQUISITOS ACADEMICOS PARA LA REALIZACION DE  \nUN  MODULO  \nCódigo: ES -ENA -551  Rev.   6, Fecha:  23-03-12  \nPág.  1 de 4  \n  \n   \n \n  \nUAGRM BUSINESS SCHOOL      \n  \n  \n \n  \n   \n  \n  \nMAESTRÍA:  \nMAESTRÍA EN ADMINISTRACIÓN DE EMPRESAS  \n(Versión 9º – Edición 10º)  \nGESTIÓN DE MARKETING   \n(ESTRATÉGICO Y OPERATIVO)  \n  \n  \n \n  \nEnero de 2022  \nSanta Cruz – Bolivia  \n \n \nANEXO  \n  \n1 \n: \n  \nFORMATO DE  \nPRESENTACION DEL PROGRAMA ANAL  \nI \nTICO  \n:']
['Página no. 2 UAGRM  Business School    Calle Gral. Saavedra Nº 137     Teléfono  3 -366814      www.uagrmbs.edu.bo   \n  E-mail: solicitudes@uagrmbs.edu.bo   \n  \n   ESPECIFICACION  \n \nREQUISITOS ACADEMICOS PARA LA REALIZACION DE  \nUN  MODULO  \nCódigo: ES -ENA -551  Rev.   6, Fecha:  23-03-12  \nPág.  2 de 4 

In [52]:
query = "Responde en español: ¿Qué es el marketing?"
docs = VectorStore.similarity_search(query=query, k=3)
#docs = loaded_VectorStore.max_marginal_relevance_search(query=query, k=5)

llm = LLM_gradioAPI(n=100) # hugging face repo
chain = load_qa_chain(llm=llm, chain_type="stuff", verbose=True)

reference = "------Also, Write in new lines the initial 10 words of each context for reference."
with get_openai_callback() as cb:
    response = chain.run(input_documents=docs, question=query)
    print(cb)
print(response)

KeyError: 'page'

### Using langcahin pypdf loader

In [76]:
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings # see also sentence-transformers
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback
from src import LLM_gradioAPI
from langchain.document_loaders import PyPDFLoader

#reader = PdfReader("PROGRAMA ANALITICO MÓDULO.pdf")
#reader = PdfReader("PatekPhilippe_P2400_Manual_Winding.pdf")
#reader = PdfReader("Philip Kotler, Gary Armstrong - Marketing_ Versión latinoamerica (2007).pdf")
#reader = PdfReader("Philip Kotler, Gary Armstrong, Sridhar Balasubramanian - Principles of Marketing-Pearson (2023).pdf")

#loader = PyPDFLoader("PROGRAMA ANALITICO MÓDULO.pdf")
loader = PyPDFLoader("Philip Kotler, Gary Armstrong - Marketing_ Versión latinoamerica (2007).pdf")


pages = loader.load_and_split()

chunks = text_splitter.split_documents(pages)


embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v1")
faiss_index = FAISS.from_documents(chunks, embedding=embeddings)


Referencias:

Página no.: 17
 xvi Prefacio
CENTRO DE RECURSOS PARA EL INSTRUCTOR: Todos los recursos para el
instructor están en inglés y protegidos por una clave, y están disponibles para su descarga en
www.pearsoneducacion.net/kotler . Para su comodidad, estos recursos también están dispo-
nibles en el CD-ROM de Recursos para el Profesor.
■Manual del instructor :Revise este recurso capítulo por capítulo o descargue todo el ma-
nual en un archivo comprimido.
■Archivo de reactivos para examen :Revise capítulo por capítulo o descargue todo el 
archivo de reactivos para examen en un archivo comprimido.
■TestGen para PC/Mac: Descargue este programa fácil de usar; ya viene incluido en las
preguntas de examen y manual del usuario de la decimoprimera edición.
■Banco de imágenes (sólo en CD) : Tenga acceso a muchas de las imágenes, anuncios e
ilustraciones incluidos en el texto. Ideal para presentaciones con PowerPoint.
■Presentaciones en PowerPoint :Cuando se trata de presentaciones en Power

In [88]:
docs = faiss_index.similarity_search("¿Qué es marketing?", k=5)
#docs = loaded_VectorStore.max_marginal_relevance_search(query=query, k=5)

llm = LLM_gradioAPI(n=2000) # hugging face repo
chain = load_qa_chain(llm=llm, chain_type="stuff", verbose=False)
response = chain.run(input_documents=docs, question=query)
#with get_openai_callback() as cb:
#    response = chain.run(input_documents=docs, question=query)
#    print(cb)

print(response)
print("\nReferencias:")
for doc in docs:
    print("\nPágina no.: " + str(doc.metadata["page"]) + "\n", doc.page_content[:150] + "...")

Loaded as API: https://ysharma-explore-llamav2-with-tgi.hf.space/ ✔
El marketing es un proceso social y administrativo mediante el cual los individuos y los grupos obtienen lo que necesitan y desean, creando e intercambiando valor con otros. En el contexto de negocios, el marketing incluye el desarrollo, la producción, la distribución, la promoción y la venta de productos o servicios que satisfagan las necesidades y deseos de los clientes. El marketing directo, por ejemplo, es una forma de marketing que se enfoca en individuos específicos y utiliza cuatro características distintivas: no es público, es inmediato y personalizado, y es interactivo. En resumen, el marketing es una estrategia utilizada para establecer relaciones con los clientes y satisfacer sus necesidades, lo que puede incluir la venta y la promoción de productos o servicios.

Referencias:

Página no.: 25
 Definición de marketing
¿Qué esel marketing? Mucha gente cree que el marketing sólo consiste en vender y hacer pu-
bl

### A draft

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings # see also sentence-transformers
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback
from src import LLM_gradioAPI
from langchain.document_loaders import PyPDFLoader # install pypdf

loader = PyPDFLoader("PROGRAMA ANALITICO MÓDULO.pdf")
#loader = PyPDFLoader("PROGRAMA ANALITICO MÓDULO.pdf")
#loader = PyPDFLoader("Philip Kotler, Gary Armstrong - Marketing_ Versión latinoamerica (2007).pdf")
#loader = PyPDFLoader("Philip Kotler, Gary Armstrong, Sridhar Balasubramanian - Principles of Marketing-Pearson (2023).pdf")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    add_start_index = False,
    )

pages = loader.load_and_split()
chunks = text_splitter.split_documents(pages)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v1")
faiss_index = FAISS.from_documents(chunks, embedding=embeddings)

In [6]:
query = "Responde en español: ¿Cuál es la experiencia de la docente?"
docs = faiss_index.similarity_search(query, k=5)
#docs = loaded_VectorStore.max_marginal_relevance_search(query=query, k=5)

llm = LLM_gradioAPI(n=2000) # hugging face repo
chain = load_qa_chain(llm=llm, chain_type="stuff", verbose=False)
response = chain.run(input_documents=docs, question=query)
#with get_openai_callback() as cb:
#    response = chain.run(input_documents=docs, question=query)
#    print(cb)

print(response)
print("\nReferencias:")
for doc in docs:
    print("\nPágina no.: " + str(doc.metadata["page"]) + "\n", doc.page_content[:150] + "...")

Loaded as API: https://ysharma-explore-llamav2-with-tgi.hf.space/ ✔
La docente tiene experiencia en el área comercial y ventas de empresas de servicio, como Principal Créditos Hipotecarios, Andueza Corredores de Seguros, y Joyerías Anahí. Además, ha trabajado en la gerencia de ventas y como consultora en temas empresariales y de marketing. También ha enseñado en pre y postgrado en UTEPSA, UAGRM Business School y UCB.

Referencias:

Página no.: 7
 o Contenido Práctico. Se basa en la aplicación del contenido teórico en los trabajos desarrollados por los 
postgraduantes así como  el estudio artícu...

Página no.: 1
 Experiencia en el área comercial y ventas de empresas de servicio como Principal Créditos Hipotecarios, 
Andueza Corredores de Seguros, Santiago -Chil...

Página no.: 7
 UAGRM  Business School    Calle Gral. Saavedra Nº 137     Teléfono  3 -366814      www.uagrmbs.edu.bo   
  E-mail: solicitudes@uagrmbs.edu.bo   
  
  ...

Página no.: 8
 UAGRM  Business School    Calle Gral. S